In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import numpy as np
import cc3d

import cv2
from PIL import Image

import pandas as pd

from tqdm import tqdm_notebook as tqdm

from skimage.measure import regionprops, label, find_contours
from scipy.spatial.distance import cdist
from scipy import stats

import scipy.ndimage as ndimage

In [ ]:
def normalize_x(image):
    return image / 127.5 - 1


def normalize_y(image):
    return image / 255


def denormalize_y(image):
    return image * 255

In [ ]:
def load_X_gray(folder_path):
    
    image_files = []

    #image_files = os.listdir(folder_path)
       
    for file in os.listdir(folder_path):
        base, ext = os.path.splitext(file)
        if ext == '.png':
            image_files.append(file)
        else :
            pass
        
    image_files.sort()
    print (image_files)
    
    img = cv2.imread(folder_path + os.sep + image_files[0], cv2.IMREAD_GRAYSCALE)
    
    #image_files = image_files[1:]
    images = np.zeros((len(image_files), img.shape[0], img.shape[1], 1), np.float32)
    for i, image_file in enumerate(image_files):
        image = cv2.imread(folder_path + os.sep + image_file, cv2.IMREAD_GRAYSCALE)
        print (image.shape)
        #image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
        image = image[:, :, np.newaxis]
        images[i] = normalize_x(image)
    return images, image_files


def load_Y_gray(folder_path, thresh = None , normalize = True):
    image_files = []
    #image_files = os.listdir(folder_path)
    
    for file in os.listdir(folder_path):
        base , ext = os.path.splitext(file)
        if ext == '.png':
            image_files.append(file)
        else:
            pass
        
    image_files.sort()
    print(image_files)
    img = cv2.imread(folder_path + os.sep + image_files[0], cv2.IMREAD_GRAYSCALE)
    images = np.zeros(
        (
            len(image_files) ,
            img.shape[0] ,
            img.shape[1] ,
            1
        ) ,
        np.float32
    )
    
    for i , image_file in enumerate(image_files):
        image = cv2.imread(
            folder_path + os.sep + image_file ,
            cv2.IMREAD_GRAYSCALE
        )
        if thresh:
            ret , image = cv2.threshold(image , thresh , 255 , cv2.THRESH_BINARY)
        image = cv2.resize(image ,(img.shape[1] ,img.shape[0]))
        image = image[ : , : , np.newaxis]
        if normalize:
            images[i] = normalize_y(image)
        else:
            images[i] = image
            
    print(images.shape)
    
    return images , image_files

def load_Y_RGB(folder_path, normalize = True):
    image_files = []
    #image_files = os.listdir(folder_path)
    
    for file in os.listdir(folder_path):
        base , ext = os.path.splitext(file)
        if ext == '.png':
            image_files.append(file)
        else:
            pass
        
    image_files.sort()
    img = cv2.imread(folder_path + os.sep + image_files[0])
    images = np.zeros((len(image_files), img.shape[0], img.shape[1], 3), np.float32)
    
    for i , image_file in enumerate(image_files):
        image = cv2.imread(folder_path + os.sep + image_file)
        image = cv2.resize(image ,(img.shape[1] ,img.shape[0]))
        if normalize:
            images[i] = normalize_y(image)
        else:
            images[i] = image
            
    print(images.shape)
    
    return images , image_files

In [ ]:
def erode_mito(imgs, erosion_iteration):
    
    diamond = ndimage.generate_binary_structure(rank=3, connectivity=1)
    eroded_mito = ndimage.binary_erosion(imgs, diamond, iterations=erosion_iteration)
    
    return eroded_mito

In [ ]:
def contract_contours(imgs, erosion_iteration):
    
    eroded_imgs = erode_mito(imgs, erosion_iteration-1).astype("uint8")
    plus_eroded_imgs = erode_mito(imgs, erosion_iteration).astype("uint8")
    
    contours_imgs = eroded_imgs - plus_eroded_imgs
    return contours_imgs

In [ ]:
mito_imgs, _ = load_Y_gray("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/devided_mito/", normalize=False)
lamellar_imgs, _ = load_Y_gray("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/merged_cristae_final/lamellar_with_postprocessing/", normalize=False)
tubular_imgs, _ = load_Y_gray("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/merged_cristae_final/tubular_with_postprocessing/", normalize=False)
connectivity = 6 
labels_mito_imgs = cc3d.connected_components((mito_imgs[:,:,:,0] == 1).astype(int), connectivity = connectivity)

In [ ]:
OUT_DIR = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_junction_002/"
os.makedirs(f"{OUT_DIR}/", exist_ok = True)

for i in tqdm(range(int(np.max(labels_mito_imgs) + 1))):

    
    if i != 0:
        target_mito_imgs = np.where(
            labels_mito_imgs[..., np.newaxis] == i,
            1,
            0
        ).astype("uint8")
        
        target_lamellar_imgs = np.where(
            target_mito_imgs > 0,
            lamellar_imgs,
            0
        )
        
        target_tubular_imgs = np.where(
            target_mito_imgs > 0,
            tubular_imgs,
            0
        )
        
        target_cristae_imgs = np.where(
            target_lamellar_imgs + target_tubular_imgs > 0,
            1,
            0
        )
        
        del target_lamellar_imgs, target_tubular_imgs
        
        #counter_imgs = extract_contours(target_mito_imgs)
        counter_imgs_matrix = target_mito_imgs.nonzero()
        
        cropped_mito_imgs = target_mito_imgs[
            max(0, np.min(counter_imgs_matrix[0]) - 5): min(labels_mito_imgs.shape[0], np.max(counter_imgs_matrix[0]) + 5),
            max(0, np.min(counter_imgs_matrix[1]) - 5): min(labels_mito_imgs.shape[1], np.max(counter_imgs_matrix[1]) + 5),
            max(0, np.min(counter_imgs_matrix[2]) - 5): min(labels_mito_imgs.shape[2], np.max(counter_imgs_matrix[2]) + 5),
            :
        ]
        
        cropped_cristae_imgs = target_cristae_imgs[
            max(0, np.min(counter_imgs_matrix[0]) - 5): min(labels_mito_imgs.shape[0], np.max(counter_imgs_matrix[0]) + 5),
            max(0, np.min(counter_imgs_matrix[1]) - 5): min(labels_mito_imgs.shape[1], np.max(counter_imgs_matrix[1]) + 5),
            max(0, np.min(counter_imgs_matrix[2]) - 5): min(labels_mito_imgs.shape[2], np.max(counter_imgs_matrix[2]) + 5),
            :            
        ]
        
        counter_2px_3px_4px_imgs = np.zeros(
            (cropped_mito_imgs.shape[0], cropped_mito_imgs.shape[1], cropped_mito_imgs.shape[2], 3),
            np.float32
        )
        
        for iteration in range(2, 5):     
            counter_eroded_imgs = contract_contours(cropped_mito_imgs[:,:,:,0], erosion_iteration=iteration).astype("uint8")
            duplicated_imgs = np.where(
                counter_eroded_imgs > 0,
                cropped_cristae_imgs[:,:,:,0],
                0
            )
            
            counter_2px_3px_4px_imgs[:,:,:,iteration-2] = duplicated_imgs
            
        os.makedirs(f"{OUT_DIR}/{str(i).zfill(4)}", exist_ok = True)
        for j in range(counter_2px_3px_4px_imgs.shape[0]):
            cv2.imwrite(
                f"{OUT_DIR}/{str(i).zfill(4)}/{str(j).zfill(4)}.png",
                counter_2px_3px_4px_imgs[j]
            )
            

In [ ]:
mito_imgs, _ = load_Y_gray("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/devided_mito/", normalize=False)
lamellar_imgs, _ = load_Y_gray("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/merged_cristae_final/lamellar_with_postprocessing/", normalize=False)
tubular_imgs, _ = load_Y_gray("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/merged_cristae_final/tubular_with_postprocessing/", normalize=False)
connectivity = 6 
labels_mito_imgs = cc3d.connected_components((mito_imgs[:,:,:,0] == 1).astype(int), connectivity = connectivity)
labels_mito_imgs = labels_mito_imgs[:, :692, :2275]

In [ ]:
OUT_DIR = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_junction_002/"
os.makedirs(f"{OUT_DIR}/", exist_ok = True)

for i in tqdm(range(int(np.max(labels_mito_imgs) + 1))):

    
    if i != 0:
        target_mito_imgs = np.where(
            labels_mito_imgs[..., np.newaxis] == i,
            1,
            0
        ).astype("uint8")
        
        target_lamellar_imgs = np.where(
            target_mito_imgs > 0,
            lamellar_imgs,
            0
        )
        
        target_tubular_imgs = np.where(
            target_mito_imgs > 0,
            tubular_imgs,
            0
        )
        
        target_cristae_imgs = np.where(
            target_lamellar_imgs + target_tubular_imgs > 0,
            1,
            0
        )
        
        del target_lamellar_imgs, target_tubular_imgs
        
        #counter_imgs = extract_contours(target_mito_imgs)
        counter_imgs_matrix = target_mito_imgs.nonzero()
        
        cropped_mito_imgs = target_mito_imgs[
            max(0, np.min(counter_imgs_matrix[0]) - 5): min(labels_mito_imgs.shape[0], np.max(counter_imgs_matrix[0]) + 5),
            max(0, np.min(counter_imgs_matrix[1]) - 5): min(labels_mito_imgs.shape[1], np.max(counter_imgs_matrix[1]) + 5),
            max(0, np.min(counter_imgs_matrix[2]) - 5): min(labels_mito_imgs.shape[2], np.max(counter_imgs_matrix[2]) + 5),
            :
        ]
        
        cropped_cristae_imgs = target_cristae_imgs[
            max(0, np.min(counter_imgs_matrix[0]) - 5): min(labels_mito_imgs.shape[0], np.max(counter_imgs_matrix[0]) + 5),
            max(0, np.min(counter_imgs_matrix[1]) - 5): min(labels_mito_imgs.shape[1], np.max(counter_imgs_matrix[1]) + 5),
            max(0, np.min(counter_imgs_matrix[2]) - 5): min(labels_mito_imgs.shape[2], np.max(counter_imgs_matrix[2]) + 5),
            :            
        ]
        
        counter_2px_3px_4px_imgs = np.zeros(
            (cropped_mito_imgs.shape[0], cropped_mito_imgs.shape[1], cropped_mito_imgs.shape[2], 3),
            np.float32
        )
        
        for iteration in range(2, 5):     
            counter_eroded_imgs = contract_contours(cropped_mito_imgs[:,:,:,0], erosion_iteration=iteration).astype("uint8")
            duplicated_imgs = np.where(
                counter_eroded_imgs > 0,
                cropped_cristae_imgs[:,:,:,0],
                0
            )
            
            counter_2px_3px_4px_imgs[:,:,:,iteration-2] = duplicated_imgs
            
        os.makedirs(f"{OUT_DIR}/{str(i).zfill(4)}", exist_ok = True)
        for j in range(counter_2px_3px_4px_imgs.shape[0]):
            cv2.imwrite(
                f"{OUT_DIR}/{str(i).zfill(4)}/{str(j).zfill(4)}.png",
                counter_2px_3px_4px_imgs[j]
            )
            

In [ ]:
mito_imgs, _ = load_Y_gray("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/devided_mito/", normalize=False)
lamellar_imgs, _ = load_Y_gray("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/merged_cristae_final/lamellar_with_postprocessing/", normalize=False)
tubular_imgs, _ = load_Y_gray("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/merged_cristae_final/tubular_with_postprocessing/", normalize=False)
connectivity = 6 
labels_mito_imgs = cc3d.connected_components((mito_imgs[:,:,:,0] == 1).astype(int), connectivity = connectivity)

In [ ]:
OUT_DIR = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/cristae_junction_002/"
os.makedirs(f"{OUT_DIR}/", exist_ok = True)

for i in tqdm(range(int(np.max(labels_mito_imgs) + 1))):

    
    if i != 0:
        target_mito_imgs = np.where(
            labels_mito_imgs[..., np.newaxis] == i,
            1,
            0
        ).astype("uint8")
        
        target_lamellar_imgs = np.where(
            target_mito_imgs > 0,
            lamellar_imgs,
            0
        )
        
        target_tubular_imgs = np.where(
            target_mito_imgs > 0,
            tubular_imgs,
            0
        )
        
        target_cristae_imgs = np.where(
            target_lamellar_imgs + target_tubular_imgs > 0,
            1,
            0
        )
        
        del target_lamellar_imgs, target_tubular_imgs
        
        #counter_imgs = extract_contours(target_mito_imgs)
        counter_imgs_matrix = target_mito_imgs.nonzero()
        
        cropped_mito_imgs = target_mito_imgs[
            max(0, np.min(counter_imgs_matrix[0]) - 5): min(labels_mito_imgs.shape[0], np.max(counter_imgs_matrix[0]) + 5),
            max(0, np.min(counter_imgs_matrix[1]) - 5): min(labels_mito_imgs.shape[1], np.max(counter_imgs_matrix[1]) + 5),
            max(0, np.min(counter_imgs_matrix[2]) - 5): min(labels_mito_imgs.shape[2], np.max(counter_imgs_matrix[2]) + 5),
            :
        ]
        
        cropped_cristae_imgs = target_cristae_imgs[
            max(0, np.min(counter_imgs_matrix[0]) - 5): min(labels_mito_imgs.shape[0], np.max(counter_imgs_matrix[0]) + 5),
            max(0, np.min(counter_imgs_matrix[1]) - 5): min(labels_mito_imgs.shape[1], np.max(counter_imgs_matrix[1]) + 5),
            max(0, np.min(counter_imgs_matrix[2]) - 5): min(labels_mito_imgs.shape[2], np.max(counter_imgs_matrix[2]) + 5),
            :            
        ]
        
        counter_2px_3px_4px_imgs = np.zeros(
            (cropped_mito_imgs.shape[0], cropped_mito_imgs.shape[1], cropped_mito_imgs.shape[2], 3),
            np.float32
        )
        
        for iteration in range(2, 5):     
            counter_eroded_imgs = contract_contours(cropped_mito_imgs[:,:,:,0], erosion_iteration=iteration).astype("uint8")
            duplicated_imgs = np.where(
                counter_eroded_imgs > 0,
                cropped_cristae_imgs[:,:,:,0],
                0
            )
            
            counter_2px_3px_4px_imgs[:,:,:,iteration-2] = duplicated_imgs
            
        os.makedirs(f"{OUT_DIR}/{str(i).zfill(4)}", exist_ok = True)
        for j in range(counter_2px_3px_4px_imgs.shape[0]):
            cv2.imwrite(
                f"{OUT_DIR}/{str(i).zfill(4)}/{str(j).zfill(4)}.png",
                counter_2px_3px_4px_imgs[j]
            )
            

In [ ]:
mito_imgs, _ = load_Y_gray("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/devided_mito/", normalize=False)
lamellar_imgs, _ = load_Y_gray("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/merged_cristae_final/lamellar_with_postprocessing/", normalize=False)
tubular_imgs, _ = load_Y_gray("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/merged_cristae_final/tubular_with_postprocessing/", normalize=False)
connectivity = 6 
labels_mito_imgs = cc3d.connected_components((mito_imgs[:,:,:,0] == 1).astype(int), connectivity = connectivity)

In [ ]:
OUT_DIR = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/cristae_junction_002/"
os.makedirs(f"{OUT_DIR}/", exist_ok = True)

for i in tqdm(range(int(np.max(labels_mito_imgs) + 1))):

    
    if i != 0:
        target_mito_imgs = np.where(
            labels_mito_imgs[..., np.newaxis] == i,
            1,
            0
        ).astype("uint8")
        
        target_lamellar_imgs = np.where(
            target_mito_imgs > 0,
            lamellar_imgs,
            0
        )
        
        target_tubular_imgs = np.where(
            target_mito_imgs > 0,
            tubular_imgs,
            0
        )
        
        target_cristae_imgs = np.where(
            target_lamellar_imgs + target_tubular_imgs > 0,
            1,
            0
        )
        
        del target_lamellar_imgs, target_tubular_imgs
        
        #counter_imgs = extract_contours(target_mito_imgs)
        counter_imgs_matrix = target_mito_imgs.nonzero()
        
        cropped_mito_imgs = target_mito_imgs[
            max(0, np.min(counter_imgs_matrix[0]) - 5): min(labels_mito_imgs.shape[0], np.max(counter_imgs_matrix[0]) + 5),
            max(0, np.min(counter_imgs_matrix[1]) - 5): min(labels_mito_imgs.shape[1], np.max(counter_imgs_matrix[1]) + 5),
            max(0, np.min(counter_imgs_matrix[2]) - 5): min(labels_mito_imgs.shape[2], np.max(counter_imgs_matrix[2]) + 5),
            :
        ]
        
        cropped_cristae_imgs = target_cristae_imgs[
            max(0, np.min(counter_imgs_matrix[0]) - 5): min(labels_mito_imgs.shape[0], np.max(counter_imgs_matrix[0]) + 5),
            max(0, np.min(counter_imgs_matrix[1]) - 5): min(labels_mito_imgs.shape[1], np.max(counter_imgs_matrix[1]) + 5),
            max(0, np.min(counter_imgs_matrix[2]) - 5): min(labels_mito_imgs.shape[2], np.max(counter_imgs_matrix[2]) + 5),
            :            
        ]
        
        counter_2px_3px_4px_imgs = np.zeros(
            (cropped_mito_imgs.shape[0], cropped_mito_imgs.shape[1], cropped_mito_imgs.shape[2], 3),
            np.float32
        )
        
        for iteration in range(2, 5):     
            counter_eroded_imgs = contract_contours(cropped_mito_imgs[:,:,:,0], erosion_iteration=iteration).astype("uint8")
            duplicated_imgs = np.where(
                counter_eroded_imgs > 0,
                cropped_cristae_imgs[:,:,:,0],
                0
            )
            
            counter_2px_3px_4px_imgs[:,:,:,iteration-2] = duplicated_imgs
            
        os.makedirs(f"{OUT_DIR}/{str(i).zfill(4)}", exist_ok = True)
        for j in range(counter_2px_3px_4px_imgs.shape[0]):
            cv2.imwrite(
                f"{OUT_DIR}/{str(i).zfill(4)}/{str(j).zfill(4)}.png",
                counter_2px_3px_4px_imgs[j]
            )
            

In [ ]:
mito_imgs, _ = load_Y_gray("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/devided_mito/", normalize=False)
lamellar_imgs, _ = load_Y_gray("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/merged_cristae_final/lamellar_with_postprocessing/", normalize=False)
tubular_imgs, _ = load_Y_gray("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/merged_cristae_final/tubular_with_postprocessing/", normalize=False)
connectivity = 6 
labels_mito_imgs = cc3d.connected_components((mito_imgs[:,:,:,0] == 1).astype(int), connectivity = connectivity)

In [ ]:
OUT_DIR = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/cristae_junction_002/"
os.makedirs(f"{OUT_DIR}/", exist_ok = True)

for i in tqdm(range(int(np.max(labels_mito_imgs) + 1))):

    
    if i != 0:
        target_mito_imgs = np.where(
            labels_mito_imgs[..., np.newaxis] == i,
            1,
            0
        ).astype("uint8")
        
        target_lamellar_imgs = np.where(
            target_mito_imgs > 0,
            lamellar_imgs,
            0
        )
        
        target_tubular_imgs = np.where(
            target_mito_imgs > 0,
            tubular_imgs,
            0
        )
        
        target_cristae_imgs = np.where(
            target_lamellar_imgs + target_tubular_imgs > 0,
            1,
            0
        )
        
        del target_lamellar_imgs, target_tubular_imgs
        
        #counter_imgs = extract_contours(target_mito_imgs)
        counter_imgs_matrix = target_mito_imgs.nonzero()
        
        cropped_mito_imgs = target_mito_imgs[
            max(0, np.min(counter_imgs_matrix[0]) - 5): min(labels_mito_imgs.shape[0], np.max(counter_imgs_matrix[0]) + 5),
            max(0, np.min(counter_imgs_matrix[1]) - 5): min(labels_mito_imgs.shape[1], np.max(counter_imgs_matrix[1]) + 5),
            max(0, np.min(counter_imgs_matrix[2]) - 5): min(labels_mito_imgs.shape[2], np.max(counter_imgs_matrix[2]) + 5),
            :
        ]
        
        cropped_cristae_imgs = target_cristae_imgs[
            max(0, np.min(counter_imgs_matrix[0]) - 5): min(labels_mito_imgs.shape[0], np.max(counter_imgs_matrix[0]) + 5),
            max(0, np.min(counter_imgs_matrix[1]) - 5): min(labels_mito_imgs.shape[1], np.max(counter_imgs_matrix[1]) + 5),
            max(0, np.min(counter_imgs_matrix[2]) - 5): min(labels_mito_imgs.shape[2], np.max(counter_imgs_matrix[2]) + 5),
            :            
        ]
        
        counter_2px_3px_4px_imgs = np.zeros(
            (cropped_mito_imgs.shape[0], cropped_mito_imgs.shape[1], cropped_mito_imgs.shape[2], 3),
            np.float32
        )
        
        for iteration in range(2, 5):     
            counter_eroded_imgs = contract_contours(cropped_mito_imgs[:,:,:,0], erosion_iteration=iteration).astype("uint8")
            duplicated_imgs = np.where(
                counter_eroded_imgs > 0,
                cropped_cristae_imgs[:,:,:,0],
                0
            )
            
            counter_2px_3px_4px_imgs[:,:,:,iteration-2] = duplicated_imgs
            
        os.makedirs(f"{OUT_DIR}/{str(i).zfill(4)}", exist_ok = True)
        for j in range(counter_2px_3px_4px_imgs.shape[0]):
            cv2.imwrite(
                f"{OUT_DIR}/{str(i).zfill(4)}/{str(j).zfill(4)}.png",
                counter_2px_3px_4px_imgs[j]
            )
            

In [ ]:
mito_imgs, _ = load_Y_gray("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/devided_mito/", normalize=False)
lamellar_imgs, _ = load_Y_gray("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_final/lamellar_with_postprocessing/", normalize=False)
tubular_imgs, _ = load_Y_gray("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_final/tubular_with_postprocessing/", normalize=False)
connectivity = 6 
labels_mito_imgs = cc3d.connected_components((mito_imgs[:,:,:,0] == 1).astype(int), connectivity = connectivity)

In [ ]:
OUT_DIR = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_junction_002/"
os.makedirs(f"{OUT_DIR}/", exist_ok = True)

for i in tqdm(range(int(np.max(labels_mito_imgs) + 1))):

    
    if i != 0:
        target_mito_imgs = np.where(
            labels_mito_imgs[..., np.newaxis] == i,
            1,
            0
        ).astype("uint8")
        
        target_lamellar_imgs = np.where(
            target_mito_imgs > 0,
            lamellar_imgs,
            0
        )
        
        target_tubular_imgs = np.where(
            target_mito_imgs > 0,
            tubular_imgs,
            0
        )
        
        target_cristae_imgs = np.where(
            target_lamellar_imgs + target_tubular_imgs > 0,
            1,
            0
        )
        
        del target_lamellar_imgs, target_tubular_imgs
        
        #counter_imgs = extract_contours(target_mito_imgs)
        counter_imgs_matrix = target_mito_imgs.nonzero()
        
        cropped_mito_imgs = target_mito_imgs[
            max(0, np.min(counter_imgs_matrix[0]) - 5): min(labels_mito_imgs.shape[0], np.max(counter_imgs_matrix[0]) + 5),
            max(0, np.min(counter_imgs_matrix[1]) - 5): min(labels_mito_imgs.shape[1], np.max(counter_imgs_matrix[1]) + 5),
            max(0, np.min(counter_imgs_matrix[2]) - 5): min(labels_mito_imgs.shape[2], np.max(counter_imgs_matrix[2]) + 5),
            :
        ]
        
        cropped_cristae_imgs = target_cristae_imgs[
            max(0, np.min(counter_imgs_matrix[0]) - 5): min(labels_mito_imgs.shape[0], np.max(counter_imgs_matrix[0]) + 5),
            max(0, np.min(counter_imgs_matrix[1]) - 5): min(labels_mito_imgs.shape[1], np.max(counter_imgs_matrix[1]) + 5),
            max(0, np.min(counter_imgs_matrix[2]) - 5): min(labels_mito_imgs.shape[2], np.max(counter_imgs_matrix[2]) + 5),
            :            
        ]
        
        counter_2px_3px_4px_imgs = np.zeros(
            (cropped_mito_imgs.shape[0], cropped_mito_imgs.shape[1], cropped_mito_imgs.shape[2], 3),
            np.float32
        )
        
        for iteration in range(2, 5):     
            counter_eroded_imgs = contract_contours(cropped_mito_imgs[:,:,:,0], erosion_iteration=iteration).astype("uint8")
            duplicated_imgs = np.where(
                counter_eroded_imgs > 0,
                cropped_cristae_imgs[:,:,:,0],
                0
            )
            
            counter_2px_3px_4px_imgs[:,:,:,iteration-2] = duplicated_imgs
            
        os.makedirs(f"{OUT_DIR}/{str(i).zfill(4)}", exist_ok = True)
        for j in range(counter_2px_3px_4px_imgs.shape[0]):
            cv2.imwrite(
                f"{OUT_DIR}/{str(i).zfill(4)}/{str(j).zfill(4)}.png",
                counter_2px_3px_4px_imgs[j]
            )
            

In [ ]:
mito_imgs, _ = load_Y_gray("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/devided_mito/", normalize=False)
lamellar_imgs, _ = load_Y_gray("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_final/lamellar_with_postprocessing/", normalize=False)
tubular_imgs, _ = load_Y_gray("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_final/tubular_with_postprocessing/", normalize=False)
connectivity = 6 
labels_mito_imgs = cc3d.connected_components((mito_imgs[:,:,:,0] == 1).astype(int), connectivity = connectivity)

In [ ]:
OUT_DIR = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_junction_002/"
os.makedirs(f"{OUT_DIR}/", exist_ok = True)

for i in tqdm(range(int(np.max(labels_mito_imgs) + 1))):

    
    if i != 0:
        target_mito_imgs = np.where(
            labels_mito_imgs[..., np.newaxis] == i,
            1,
            0
        ).astype("uint8")
        
        target_lamellar_imgs = np.where(
            target_mito_imgs > 0,
            lamellar_imgs,
            0
        )
        
        target_tubular_imgs = np.where(
            target_mito_imgs > 0,
            tubular_imgs,
            0
        )
        
        target_cristae_imgs = np.where(
            target_lamellar_imgs + target_tubular_imgs > 0,
            1,
            0
        )
        
        del target_lamellar_imgs, target_tubular_imgs
        
        #counter_imgs = extract_contours(target_mito_imgs)
        counter_imgs_matrix = target_mito_imgs.nonzero()
        
        cropped_mito_imgs = target_mito_imgs[
            max(0, np.min(counter_imgs_matrix[0]) - 5): min(labels_mito_imgs.shape[0], np.max(counter_imgs_matrix[0]) + 5),
            max(0, np.min(counter_imgs_matrix[1]) - 5): min(labels_mito_imgs.shape[1], np.max(counter_imgs_matrix[1]) + 5),
            max(0, np.min(counter_imgs_matrix[2]) - 5): min(labels_mito_imgs.shape[2], np.max(counter_imgs_matrix[2]) + 5),
            :
        ]
        
        cropped_cristae_imgs = target_cristae_imgs[
            max(0, np.min(counter_imgs_matrix[0]) - 5): min(labels_mito_imgs.shape[0], np.max(counter_imgs_matrix[0]) + 5),
            max(0, np.min(counter_imgs_matrix[1]) - 5): min(labels_mito_imgs.shape[1], np.max(counter_imgs_matrix[1]) + 5),
            max(0, np.min(counter_imgs_matrix[2]) - 5): min(labels_mito_imgs.shape[2], np.max(counter_imgs_matrix[2]) + 5),
            :            
        ]
        
        counter_2px_3px_4px_imgs = np.zeros(
            (cropped_mito_imgs.shape[0], cropped_mito_imgs.shape[1], cropped_mito_imgs.shape[2], 3),
            np.float32
        )
        
        for iteration in range(2, 5):     
            counter_eroded_imgs = contract_contours(cropped_mito_imgs[:,:,:,0], erosion_iteration=iteration).astype("uint8")
            duplicated_imgs = np.where(
                counter_eroded_imgs > 0,
                cropped_cristae_imgs[:,:,:,0],
                0
            )
            
            counter_2px_3px_4px_imgs[:,:,:,iteration-2] = duplicated_imgs
            
        os.makedirs(f"{OUT_DIR}/{str(i).zfill(4)}", exist_ok = True)
        for j in range(counter_2px_3px_4px_imgs.shape[0]):
            cv2.imwrite(
                f"{OUT_DIR}/{str(i).zfill(4)}/{str(j).zfill(4)}.png",
                counter_2px_3px_4px_imgs[j]
            )
            

In [ ]:
mito_imgs, _ = load_Y_gray("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/devided_mito/", normalize=False)
lamellar_imgs, _ = load_Y_gray("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/merged_cristae_final/lamellar_with_postprocessing/", normalize=False)
tubular_imgs, _ = load_Y_gray("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/merged_cristae_final/tubular_with_postprocessing/", normalize=False)
connectivity = 6
labels_mito_imgs = cc3d.connected_components((mito_imgs[:,:,:,0] == 1).astype(int), connectivity = connectivity)

In [ ]:
OUT_DIR = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/cristae_junction_002/"
os.makedirs(f"{OUT_DIR}/", exist_ok = True)

for i in tqdm(range(int(np.max(labels_mito_imgs) + 1))):

    
    if i != 0:
        target_mito_imgs = np.where(
            labels_mito_imgs[..., np.newaxis] == i,
            1,
            0
        ).astype("uint8")
        
        target_lamellar_imgs = np.where(
            target_mito_imgs > 0,
            lamellar_imgs,
            0
        )
        
        target_tubular_imgs = np.where(
            target_mito_imgs > 0,
            tubular_imgs,
            0
        )
        
        target_cristae_imgs = np.where(
            target_lamellar_imgs + target_tubular_imgs > 0,
            1,
            0
        )
        
        del target_lamellar_imgs, target_tubular_imgs
        
        #counter_imgs = extract_contours(target_mito_imgs)
        counter_imgs_matrix = target_mito_imgs.nonzero()
        
        cropped_mito_imgs = target_mito_imgs[
            max(0, np.min(counter_imgs_matrix[0]) - 5): min(labels_mito_imgs.shape[0], np.max(counter_imgs_matrix[0]) + 5),
            max(0, np.min(counter_imgs_matrix[1]) - 5): min(labels_mito_imgs.shape[1], np.max(counter_imgs_matrix[1]) + 5),
            max(0, np.min(counter_imgs_matrix[2]) - 5): min(labels_mito_imgs.shape[2], np.max(counter_imgs_matrix[2]) + 5),
            :
        ]
        
        cropped_cristae_imgs = target_cristae_imgs[
            max(0, np.min(counter_imgs_matrix[0]) - 5): min(labels_mito_imgs.shape[0], np.max(counter_imgs_matrix[0]) + 5),
            max(0, np.min(counter_imgs_matrix[1]) - 5): min(labels_mito_imgs.shape[1], np.max(counter_imgs_matrix[1]) + 5),
            max(0, np.min(counter_imgs_matrix[2]) - 5): min(labels_mito_imgs.shape[2], np.max(counter_imgs_matrix[2]) + 5),
            :            
        ]
        
        counter_2px_3px_4px_imgs = np.zeros(
            (cropped_mito_imgs.shape[0], cropped_mito_imgs.shape[1], cropped_mito_imgs.shape[2], 3),
            np.float32
        )
        
        for iteration in range(2, 5):     
            counter_eroded_imgs = contract_contours(cropped_mito_imgs[:,:,:,0], erosion_iteration=iteration).astype("uint8")
            duplicated_imgs = np.where(
                counter_eroded_imgs > 0,
                cropped_cristae_imgs[:,:,:,0],
                0
            )
            
            counter_2px_3px_4px_imgs[:,:,:,iteration-2] = duplicated_imgs
            
        os.makedirs(f"{OUT_DIR}/{str(i).zfill(4)}", exist_ok = True)
        for j in range(counter_2px_3px_4px_imgs.shape[0]):
            cv2.imwrite(
                f"{OUT_DIR}/{str(i).zfill(4)}/{str(j).zfill(4)}.png",
                counter_2px_3px_4px_imgs[j]
            )
            

In [ ]:
mito_imgs, _ = load_Y_gray("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/devided_mito/", normalize=False)
lamellar_imgs, _ = load_Y_gray("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/merged_cristae_final/lamellar_with_postprocessing/", normalize=False)
tubular_imgs, _ = load_Y_gray("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/merged_cristae_final/tubular_with_postprocessing/", normalize=False)
connectivity = 6 
labels_mito_imgs = cc3d.connected_components((mito_imgs[:,:,:,0] == 1).astype(int), connectivity = connectivity)

In [ ]:
OUT_DIR = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/cristae_junction_002/"
os.makedirs(f"{OUT_DIR}/", exist_ok = True)

for i in tqdm(range(int(np.max(labels_mito_imgs) + 1))):

    
    if i != 0:
        target_mito_imgs = np.where(
            labels_mito_imgs[..., np.newaxis] == i,
            1,
            0
        ).astype("uint8")
        
        target_lamellar_imgs = np.where(
            target_mito_imgs > 0,
            lamellar_imgs,
            0
        )
        
        target_tubular_imgs = np.where(
            target_mito_imgs > 0,
            tubular_imgs,
            0
        )
        
        target_cristae_imgs = np.where(
            target_lamellar_imgs + target_tubular_imgs > 0,
            1,
            0
        )
        
        del target_lamellar_imgs, target_tubular_imgs
        
        #counter_imgs = extract_contours(target_mito_imgs)
        counter_imgs_matrix = target_mito_imgs.nonzero()
        
        cropped_mito_imgs = target_mito_imgs[
            max(0, np.min(counter_imgs_matrix[0]) - 5): min(labels_mito_imgs.shape[0], np.max(counter_imgs_matrix[0]) + 5),
            max(0, np.min(counter_imgs_matrix[1]) - 5): min(labels_mito_imgs.shape[1], np.max(counter_imgs_matrix[1]) + 5),
            max(0, np.min(counter_imgs_matrix[2]) - 5): min(labels_mito_imgs.shape[2], np.max(counter_imgs_matrix[2]) + 5),
            :
        ]
        
        cropped_cristae_imgs = target_cristae_imgs[
            max(0, np.min(counter_imgs_matrix[0]) - 5): min(labels_mito_imgs.shape[0], np.max(counter_imgs_matrix[0]) + 5),
            max(0, np.min(counter_imgs_matrix[1]) - 5): min(labels_mito_imgs.shape[1], np.max(counter_imgs_matrix[1]) + 5),
            max(0, np.min(counter_imgs_matrix[2]) - 5): min(labels_mito_imgs.shape[2], np.max(counter_imgs_matrix[2]) + 5),
            :            
        ]
        
        counter_2px_3px_4px_imgs = np.zeros(
            (cropped_mito_imgs.shape[0], cropped_mito_imgs.shape[1], cropped_mito_imgs.shape[2], 3),
            np.float32
        )
        
        for iteration in range(2, 5):     
            counter_eroded_imgs = contract_contours(cropped_mito_imgs[:,:,:,0], erosion_iteration=iteration).astype("uint8")
            duplicated_imgs = np.where(
                counter_eroded_imgs > 0,
                cropped_cristae_imgs[:,:,:,0],
                0
            )
            
            counter_2px_3px_4px_imgs[:,:,:,iteration-2] = duplicated_imgs
            
        os.makedirs(f"{OUT_DIR}/{str(i).zfill(4)}", exist_ok = True)
        for j in range(counter_2px_3px_4px_imgs.shape[0]):
            cv2.imwrite(
                f"{OUT_DIR}/{str(i).zfill(4)}/{str(j).zfill(4)}.png",
                counter_2px_3px_4px_imgs[j]
            )
            

In [ ]:
mito_imgs, _ = load_Y_gray("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/devided_mito/", normalize=False)
lamellar_imgs, _ = load_Y_gray("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/merged_cristae_final/lamellar_with_postprocessing/", normalize=False)
tubular_imgs, _ = load_Y_gray("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/merged_cristae_final/tubular_with_postprocessing/", normalize=False)
connectivity = 6
labels_mito_imgs = cc3d.connected_components((mito_imgs[:,:,:,0] == 1).astype(int), connectivity = connectivity)

In [ ]:
OUT_DIR = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/cristae_junction_002/"
os.makedirs(f"{OUT_DIR}/", exist_ok = True)

for i in tqdm(range(int(np.max(labels_mito_imgs) + 1))):

    
    if i != 0:
        target_mito_imgs = np.where(
            labels_mito_imgs[..., np.newaxis] == i,
            1,
            0
        ).astype("uint8")
        
        target_lamellar_imgs = np.where(
            target_mito_imgs > 0,
            lamellar_imgs,
            0
        )
        
        target_tubular_imgs = np.where(
            target_mito_imgs > 0,
            tubular_imgs,
            0
        )
        
        target_cristae_imgs = np.where(
            target_lamellar_imgs + target_tubular_imgs > 0,
            1,
            0
        )
        
        del target_lamellar_imgs, target_tubular_imgs
        
        #counter_imgs = extract_contours(target_mito_imgs)
        counter_imgs_matrix = target_mito_imgs.nonzero()
        
        cropped_mito_imgs = target_mito_imgs[
            max(0, np.min(counter_imgs_matrix[0]) - 5): min(labels_mito_imgs.shape[0], np.max(counter_imgs_matrix[0]) + 5),
            max(0, np.min(counter_imgs_matrix[1]) - 5): min(labels_mito_imgs.shape[1], np.max(counter_imgs_matrix[1]) + 5),
            max(0, np.min(counter_imgs_matrix[2]) - 5): min(labels_mito_imgs.shape[2], np.max(counter_imgs_matrix[2]) + 5),
            :
        ]
        
        cropped_cristae_imgs = target_cristae_imgs[
            max(0, np.min(counter_imgs_matrix[0]) - 5): min(labels_mito_imgs.shape[0], np.max(counter_imgs_matrix[0]) + 5),
            max(0, np.min(counter_imgs_matrix[1]) - 5): min(labels_mito_imgs.shape[1], np.max(counter_imgs_matrix[1]) + 5),
            max(0, np.min(counter_imgs_matrix[2]) - 5): min(labels_mito_imgs.shape[2], np.max(counter_imgs_matrix[2]) + 5),
            :            
        ]
        
        counter_2px_3px_4px_imgs = np.zeros(
            (cropped_mito_imgs.shape[0], cropped_mito_imgs.shape[1], cropped_mito_imgs.shape[2], 3),
            np.float32
        )
        
        for iteration in range(2, 5):     
            counter_eroded_imgs = contract_contours(cropped_mito_imgs[:,:,:,0], erosion_iteration=iteration).astype("uint8")
            duplicated_imgs = np.where(
                counter_eroded_imgs > 0,
                cropped_cristae_imgs[:,:,:,0],
                0
            )
            
            counter_2px_3px_4px_imgs[:,:,:,iteration-2] = duplicated_imgs
            
        os.makedirs(f"{OUT_DIR}/{str(i).zfill(4)}", exist_ok = True)
        for j in range(counter_2px_3px_4px_imgs.shape[0]):
            cv2.imwrite(
                f"{OUT_DIR}/{str(i).zfill(4)}/{str(j).zfill(4)}.png",
                counter_2px_3px_4px_imgs[j]
            )
            

In [ ]:
def Quantify_Mito_(imgs):
    S = 0
    for i in range(imgs.shape[0]):
        if np.max(imgs[i]) > 0:
            contours, hierarchy = cv2.findContours(imgs[i],
                                       cv2.RETR_TREE,
                                       cv2.CHAIN_APPROX_SIMPLE)
            img = np.zeros((imgs.shape[1], imgs.shape[2], 1), np.float32)
            line = cv2.drawContours(img, contours, -1, (1,0,0), 1)
        
            x = imgs.shape[1]
            y = imgs.shape[2]
        
            counter = line[:,:,0]
            #print(line.shape, counter.shape)
            counter = cv2.line(counter,(0,y-1),(x,y-1),(0,0,0),thickness=1)
            counter = cv2.line(counter,(0,1),(x,1),(0,0,0),1)
            counter = cv2.line(counter,(1,0),(x-1,0),(0,0,0),1)
            counter = cv2.line(counter,(1,y),(x-1,y),(0,0,0),1)
        
            #quantified_line = quantitative_gap(line[:,:,0])
            quantified_line = np.sum(line[:,:,0])
            S += quantified_line
    return S

In [ ]:
mito_imgs, _ = load_Y_gray("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_001/annotations/cropped_001/devided_mito/", normalize=False)
connectivity = 6 # 縦横下
labels_mito_imgs = cc3d.connected_components((mito_imgs[:,:,:,0] == 1).astype(int), connectivity = connectivity)

result_dct = {}
for i in tqdm(range(int(np.max(labels_mito_imgs) + 1))):

    
    if i != 0:
        target_mito_imgs = np.where(
            labels_mito_imgs[..., np.newaxis] == i,
            1,
            0
        ).astype("uint8")
        

        counter_imgs_matrix = target_mito_imgs.nonzero()
        
        cropped_mito_imgs = target_mito_imgs[
            max(0, np.min(counter_imgs_matrix[0]) - 5): min(labels_mito_imgs.shape[0], np.max(counter_imgs_matrix[0]) + 5),
            max(0, np.min(counter_imgs_matrix[1]) - 5): min(labels_mito_imgs.shape[1], np.max(counter_imgs_matrix[1]) + 5),
            max(0, np.min(counter_imgs_matrix[2]) - 5): min(labels_mito_imgs.shape[2], np.max(counter_imgs_matrix[2]) + 5),
            :
        ]

        
        counter_eroded_imgs = erode_mito(cropped_mito_imgs[:,:,:,0], erosion_iteration=3).astype("uint8")
        
        S = Quantify_Mito_(counter_eroded_imgs)
        
        result_dct.update({
            i : {
                "imm" : S
            }
        })


df = pd.DataFrame(
    index=[], 
    columns=['mito_No', 'imm']
)

for key, values in result_dct.items():
        df = df.append(
            pd.Series(
                {
                    'mito_No' : key,
                    'imm' : values['imm']
                }
            ),  ignore_index = True
        )  
        
os.makedirs("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_001/data/cropped_001", exist_ok = True)

df.to_csv('/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_001/data/cropped_001/imm_001.csv')


In [ ]:
mito_imgs, _ = load_Y_gray("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/devided_mito/", normalize=False)
connectivity = 6 # 縦横下
labels_mito_imgs = cc3d.connected_components((mito_imgs[:,:,:,0] == 1).astype(int), connectivity = connectivity)

result_dct = {}
for i in tqdm(range(int(np.max(labels_mito_imgs) + 1))):

    
    if i != 0:
        target_mito_imgs = np.where(
            labels_mito_imgs[..., np.newaxis] == i,
            1,
            0
        ).astype("uint8")
        

        counter_imgs_matrix = target_mito_imgs.nonzero()
        
        cropped_mito_imgs = target_mito_imgs[
            max(0, np.min(counter_imgs_matrix[0]) - 5): min(labels_mito_imgs.shape[0], np.max(counter_imgs_matrix[0]) + 5),
            max(0, np.min(counter_imgs_matrix[1]) - 5): min(labels_mito_imgs.shape[1], np.max(counter_imgs_matrix[1]) + 5),
            max(0, np.min(counter_imgs_matrix[2]) - 5): min(labels_mito_imgs.shape[2], np.max(counter_imgs_matrix[2]) + 5),
            :
        ]

        
        counter_eroded_imgs = erode_mito(cropped_mito_imgs[:,:,:,0], erosion_iteration=3).astype("uint8")
        
        S = Quantify_Mito_(counter_eroded_imgs)
        
        result_dct.update({
            i : {
                "imm" : S
            }
        })


df = pd.DataFrame(
    index=[], 
    columns=['mito_No', 'imm']
)

for key, values in result_dct.items():
        df = df.append(
            pd.Series(
                {
                    'mito_No' : key,
                    'imm' : values['imm']
                }
            ),  ignore_index = True
        )  
        
os.makedirs("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/data/cropped_002", exist_ok = True)

df.to_csv('/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/data/cropped_002/imm_001.csv')

In [ ]:
mito_imgs, _ = load_Y_gray("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/devided_mito/", normalize=False)
connectivity = 6 # 縦横下
labels_mito_imgs = cc3d.connected_components((mito_imgs[:,:,:,0] == 1).astype(int), connectivity = connectivity)

result_dct = {}
for i in tqdm(range(int(np.max(labels_mito_imgs) + 1))):

    
    if i != 0:
        target_mito_imgs = np.where(
            labels_mito_imgs[..., np.newaxis] == i,
            1,
            0
        ).astype("uint8")
        

        counter_imgs_matrix = target_mito_imgs.nonzero()
        
        cropped_mito_imgs = target_mito_imgs[
            max(0, np.min(counter_imgs_matrix[0]) - 5): min(labels_mito_imgs.shape[0], np.max(counter_imgs_matrix[0]) + 5),
            max(0, np.min(counter_imgs_matrix[1]) - 5): min(labels_mito_imgs.shape[1], np.max(counter_imgs_matrix[1]) + 5),
            max(0, np.min(counter_imgs_matrix[2]) - 5): min(labels_mito_imgs.shape[2], np.max(counter_imgs_matrix[2]) + 5),
            :
        ]

        
        counter_eroded_imgs = erode_mito(cropped_mito_imgs[:,:,:,0], erosion_iteration=3).astype("uint8")
        
        S = Quantify_Mito_(counter_eroded_imgs)
        
        result_dct.update({
            i : {
                "imm" : S
            }
        })


df = pd.DataFrame(
    index=[], 
    columns=['mito_No', 'imm']
)

for key, values in result_dct.items():
        df = df.append(
            pd.Series(
                {
                    'mito_No' : key,
                    'imm' : values['imm']
                }
            ),  ignore_index = True
        )  
        
os.makedirs("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/data/cropped_003", exist_ok = True)

df.to_csv('/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/data/cropped_003/imm_001.csv')

In [ ]:
mito_imgs, _ = load_Y_gray("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/devided_mito/", normalize=False)
connectivity = 6 # 縦横下
labels_mito_imgs = cc3d.connected_components((mito_imgs[:,:,:,0] == 1).astype(int), connectivity = connectivity)

result_dct = {}
for i in tqdm(range(int(np.max(labels_mito_imgs) + 1))):

    
    if i != 0:
        target_mito_imgs = np.where(
            labels_mito_imgs[..., np.newaxis] == i,
            1,
            0
        ).astype("uint8")
        

        counter_imgs_matrix = target_mito_imgs.nonzero()
        
        cropped_mito_imgs = target_mito_imgs[
            max(0, np.min(counter_imgs_matrix[0]) - 5): min(labels_mito_imgs.shape[0], np.max(counter_imgs_matrix[0]) + 5),
            max(0, np.min(counter_imgs_matrix[1]) - 5): min(labels_mito_imgs.shape[1], np.max(counter_imgs_matrix[1]) + 5),
            max(0, np.min(counter_imgs_matrix[2]) - 5): min(labels_mito_imgs.shape[2], np.max(counter_imgs_matrix[2]) + 5),
            :
        ]

        
        counter_eroded_imgs = erode_mito(cropped_mito_imgs[:,:,:,0], erosion_iteration=3).astype("uint8")
        
        S = Quantify_Mito_(counter_eroded_imgs)
        
        result_dct.update({
            i : {
                "imm" : S
            }
        })


df = pd.DataFrame(
    index=[], 
    columns=['mito_No', 'imm']
)

for key, values in result_dct.items():
        df = df.append(
            pd.Series(
                {
                    'mito_No' : key,
                    'imm' : values['imm']
                }
            ),  ignore_index = True
        )  
        
os.makedirs("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/data/cropped_004", exist_ok = True)

df.to_csv('/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/data/cropped_004/imm_001.csv')

In [ ]:
mito_imgs, _ = load_Y_gray("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/devided_mito/", normalize=False)
connectivity = 6 # 縦横下
labels_mito_imgs = cc3d.connected_components((mito_imgs[:,:,:,0] == 1).astype(int), connectivity = connectivity)

result_dct = {}
for i in tqdm(range(int(np.max(labels_mito_imgs) + 1))):

    
    if i != 0:
        target_mito_imgs = np.where(
            labels_mito_imgs[..., np.newaxis] == i,
            1,
            0
        ).astype("uint8")
        

        counter_imgs_matrix = target_mito_imgs.nonzero()
        
        cropped_mito_imgs = target_mito_imgs[
            max(0, np.min(counter_imgs_matrix[0]) - 5): min(labels_mito_imgs.shape[0], np.max(counter_imgs_matrix[0]) + 5),
            max(0, np.min(counter_imgs_matrix[1]) - 5): min(labels_mito_imgs.shape[1], np.max(counter_imgs_matrix[1]) + 5),
            max(0, np.min(counter_imgs_matrix[2]) - 5): min(labels_mito_imgs.shape[2], np.max(counter_imgs_matrix[2]) + 5),
            :
        ]

        
        counter_eroded_imgs = erode_mito(cropped_mito_imgs[:,:,:,0], erosion_iteration=3).astype("uint8")
        
        S = Quantify_Mito_(counter_eroded_imgs)
        
        result_dct.update({
            i : {
                "imm" : S
            }
        })


df = pd.DataFrame(
    index=[], 
    columns=['mito_No', 'imm']
)

for key, values in result_dct.items():
        df = df.append(
            pd.Series(
                {
                    'mito_No' : key,
                    'imm' : values['imm']
                }
            ),  ignore_index = True
        )  
        
os.makedirs("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/data/cropped_005", exist_ok = True)

df.to_csv('/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/data/cropped_005/imm_001.csv')

In [ ]:
mito_imgs, _ = load_Y_gray("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/devided_mito/", normalize=False)
connectivity = 6 # 縦横下
labels_mito_imgs = cc3d.connected_components((mito_imgs[:,:,:,0] == 1).astype(int), connectivity = connectivity)

result_dct = {}
for i in tqdm(range(int(np.max(labels_mito_imgs) + 1))):

    
    if i != 0:
        target_mito_imgs = np.where(
            labels_mito_imgs[..., np.newaxis] == i,
            1,
            0
        ).astype("uint8")
        

        counter_imgs_matrix = target_mito_imgs.nonzero()
        
        cropped_mito_imgs = target_mito_imgs[
            max(0, np.min(counter_imgs_matrix[0]) - 5): min(labels_mito_imgs.shape[0], np.max(counter_imgs_matrix[0]) + 5),
            max(0, np.min(counter_imgs_matrix[1]) - 5): min(labels_mito_imgs.shape[1], np.max(counter_imgs_matrix[1]) + 5),
            max(0, np.min(counter_imgs_matrix[2]) - 5): min(labels_mito_imgs.shape[2], np.max(counter_imgs_matrix[2]) + 5),
            :
        ]

        
        counter_eroded_imgs = erode_mito(cropped_mito_imgs[:,:,:,0], erosion_iteration=3).astype("uint8")
        
        S = Quantify_Mito_(counter_eroded_imgs)
        
        result_dct.update({
            i : {
                "imm" : S
            }
        })


df = pd.DataFrame(
    index=[], 
    columns=['mito_No', 'imm']
)

for key, values in result_dct.items():
        df = df.append(
            pd.Series(
                {
                    'mito_No' : key,
                    'imm' : values['imm']
                }
            ),  ignore_index = True
        )  
        
os.makedirs("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/data/cropped_001", exist_ok = True)

df.to_csv('/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/data/cropped_001/imm_001.csv')


In [ ]:
mito_imgs, _ = load_Y_gray("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/devided_mito/", normalize=False)
connectivity = 6 # 縦横下
labels_mito_imgs = cc3d.connected_components((mito_imgs[:,:,:,0] == 1).astype(int), connectivity = connectivity)

result_dct = {}
for i in tqdm(range(int(np.max(labels_mito_imgs) + 1))):

    
    if i != 0:
        target_mito_imgs = np.where(
            labels_mito_imgs[..., np.newaxis] == i,
            1,
            0
        ).astype("uint8")
        

        counter_imgs_matrix = target_mito_imgs.nonzero()
        
        cropped_mito_imgs = target_mito_imgs[
            max(0, np.min(counter_imgs_matrix[0]) - 5): min(labels_mito_imgs.shape[0], np.max(counter_imgs_matrix[0]) + 5),
            max(0, np.min(counter_imgs_matrix[1]) - 5): min(labels_mito_imgs.shape[1], np.max(counter_imgs_matrix[1]) + 5),
            max(0, np.min(counter_imgs_matrix[2]) - 5): min(labels_mito_imgs.shape[2], np.max(counter_imgs_matrix[2]) + 5),
            :
        ]

        
        counter_eroded_imgs = contract_contours(cropped_mito_imgs[:,:,:,0], erosion_iteration=3).astype("uint8")
        result_dct.update({
            i : {
                "imm" : np.sum(counter_eroded_imgs)
            }
        })

df = pd.DataFrame(
    index=[], 
    columns=['mito_No', 'imm']
)

for key, values in result_dct.items():
        df = df.append(
            pd.Series(
                {
                    'mito_No' : key,
                    'imm' : values['imm']
                }
            ),  ignore_index = True
        )  
        
os.makedirs("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/data/cropped_002", exist_ok = True)

df.to_csv('/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/data/cropped_002/imm_001.csv')


In [ ]:
mito_imgs, _ = load_Y_gray("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/devided_mito/", normalize=False)
connectivity = 6 # 縦横下
labels_mito_imgs = cc3d.connected_components((mito_imgs[:,:,:,0] == 1).astype(int), connectivity = connectivity)

result_dct = {}
for i in tqdm(range(int(np.max(labels_mito_imgs) + 1))):

    
    if i != 0:
        target_mito_imgs = np.where(
            labels_mito_imgs[..., np.newaxis] == i,
            1,
            0
        ).astype("uint8")
        

        counter_imgs_matrix = target_mito_imgs.nonzero()
        
        cropped_mito_imgs = target_mito_imgs[
            max(0, np.min(counter_imgs_matrix[0]) - 5): min(labels_mito_imgs.shape[0], np.max(counter_imgs_matrix[0]) + 5),
            max(0, np.min(counter_imgs_matrix[1]) - 5): min(labels_mito_imgs.shape[1], np.max(counter_imgs_matrix[1]) + 5),
            max(0, np.min(counter_imgs_matrix[2]) - 5): min(labels_mito_imgs.shape[2], np.max(counter_imgs_matrix[2]) + 5),
            :
        ]

        
        counter_eroded_imgs = erode_mito(cropped_mito_imgs[:,:,:,0], erosion_iteration=3).astype("uint8")
        
        S = Quantify_Mito_(counter_eroded_imgs)
        
        result_dct.update({
            i : {
                "imm" : S
            }
        })


df = pd.DataFrame(
    index=[], 
    columns=['mito_No', 'imm']
)

for key, values in result_dct.items():
        df = df.append(
            pd.Series(
                {
                    'mito_No' : key,
                    'imm' : values['imm']
                }
            ),  ignore_index = True
        )  
        
os.makedirs("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/data/cropped_003", exist_ok = True)

df.to_csv('/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/data/cropped_003/imm_001.csv')


In [ ]:
mito_imgs, _ = load_Y_gray("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/devided_mito/", normalize=False)
connectivity = 6 # 縦横下
labels_mito_imgs = cc3d.connected_components((mito_imgs[:,:,:,0] == 1).astype(int), connectivity = connectivity)

result_dct = {}
for i in tqdm(range(int(np.max(labels_mito_imgs) + 1))):

    
    if i != 0:
        target_mito_imgs = np.where(
            labels_mito_imgs[..., np.newaxis] == i,
            1,
            0
        ).astype("uint8")
        

        counter_imgs_matrix = target_mito_imgs.nonzero()
        
        cropped_mito_imgs = target_mito_imgs[
            max(0, np.min(counter_imgs_matrix[0]) - 5): min(labels_mito_imgs.shape[0], np.max(counter_imgs_matrix[0]) + 5),
            max(0, np.min(counter_imgs_matrix[1]) - 5): min(labels_mito_imgs.shape[1], np.max(counter_imgs_matrix[1]) + 5),
            max(0, np.min(counter_imgs_matrix[2]) - 5): min(labels_mito_imgs.shape[2], np.max(counter_imgs_matrix[2]) + 5),
            :
        ]

        
        counter_eroded_imgs = contract_contours(cropped_mito_imgs[:,:,:,0], erosion_iteration=3).astype("uint8")
        result_dct.update({
            i : {
                "imm" : np.sum(counter_eroded_imgs)
            }
        })

df = pd.DataFrame(
    index=[], 
    columns=['mito_No', 'imm']
)

for key, values in result_dct.items():
        df = df.append(
            pd.Series(
                {
                    'mito_No' : key,
                    'imm' : values['imm']
                }
            ),  ignore_index = True
        )  
        
os.makedirs("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/data/cropped_004", exist_ok = True)

df.to_csv('/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/data/cropped_004/imm_001.csv')


In [ ]:
mito_imgs, _ = load_Y_gray("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/devided_mito/", normalize=False)
connectivity = 6 # 縦横下
labels_mito_imgs = cc3d.connected_components((mito_imgs[:,:,:,0] == 1).astype(int), connectivity = connectivity)

result_dct = {}
for i in tqdm(range(int(np.max(labels_mito_imgs) + 1))):

    
    if i != 0:
        target_mito_imgs = np.where(
            labels_mito_imgs[..., np.newaxis] == i,
            1,
            0
        ).astype("uint8")
        

        counter_imgs_matrix = target_mito_imgs.nonzero()
        
        cropped_mito_imgs = target_mito_imgs[
            max(0, np.min(counter_imgs_matrix[0]) - 5): min(labels_mito_imgs.shape[0], np.max(counter_imgs_matrix[0]) + 5),
            max(0, np.min(counter_imgs_matrix[1]) - 5): min(labels_mito_imgs.shape[1], np.max(counter_imgs_matrix[1]) + 5),
            max(0, np.min(counter_imgs_matrix[2]) - 5): min(labels_mito_imgs.shape[2], np.max(counter_imgs_matrix[2]) + 5),
            :
        ]

        
        counter_eroded_imgs = erode_mito(cropped_mito_imgs[:,:,:,0], erosion_iteration=3).astype("uint8")
        
        S = Quantify_Mito_(counter_eroded_imgs)
        
        result_dct.update({
            i : {
                "imm" : S
            }
        })


df = pd.DataFrame(
    index=[], 
    columns=['mito_No', 'imm']
)

for key, values in result_dct.items():
        df = df.append(
            pd.Series(
                {
                    'mito_No' : key,
                    'imm' : values['imm']
                }
            ),  ignore_index = True
        )  
        
os.makedirs("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/data/cropped_005", exist_ok = True)

df.to_csv('/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/data/cropped_005/imm_001.csv')


In [ ]:
cj_2px_size = []
cj_3px_size = []
cj_4px_size = []

In [ ]:
out_dir = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_junction_002/"

mito_cj_lst  = list()

connectivity = 26
cnt = 0

for folder in tqdm(sorted(os.listdir(out_dir))):
    if folder == ".DS_Store":
        continue
    cj_imgs, _ = load_Y_RGB(f"{out_dir}/{folder}", normalize=False)
    
    _lst = [0] * 3
    
    cj_2px = cc3d.connected_components((cj_imgs[:,:,:,0] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_2px) + 1):
        cj_2px_size.append(
            np.sum(
                np.where(
                    cj_2px == i,
                    1,
                    0
                )
            )
        )
        
        _lst[0] += cj_2px_size[-1]
        
    cj_3px = cc3d.connected_components((cj_imgs[:,:,:,1] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_3px) + 1):
        cj_3px_size.append(
            np.sum(
                np.where(
                    cj_3px == i,
                    1,
                    0
                )
            )
        )
        
        _lst[1] += cj_3px_size[-1]
        
    cj_4px = cc3d.connected_components((cj_imgs[:,:,:,2] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_4px) + 1):
        cj_4px_size.append(
            np.sum(
                np.where(
                    cj_4px == i,
                    1,
                    0
                )
            )
        )
        
        _lst[2] += cj_4px_size[-1]
        
        
    cnt += 1
    mito_cj_lst.append({folder : _lst})
    

In [ ]:
out_dir = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/cristae_junction_002/"

mito_cj_lst  = list()

connectivity = 26
cnt = 0

for folder in tqdm(sorted(os.listdir(out_dir))):
    if folder == ".DS_Store":
        continue
    cj_imgs, _ = load_Y_RGB(f"{out_dir}/{folder}", normalize=False)
    
    _lst = [0] * 3
    
    cj_2px = cc3d.connected_components((cj_imgs[:,:,:,0] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_2px) + 1):
        cj_2px_size.append(
            np.sum(
                np.where(
                    cj_2px == i,
                    1,
                    0
                )
            )
        )
        
        _lst[0] += cj_2px_size[-1]
        
    cj_3px = cc3d.connected_components((cj_imgs[:,:,:,1] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_3px) + 1):
        cj_3px_size.append(
            np.sum(
                np.where(
                    cj_3px == i,
                    1,
                    0
                )
            )
        )
        
        _lst[1] += cj_3px_size[-1]
        
    cj_4px = cc3d.connected_components((cj_imgs[:,:,:,2] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_4px) + 1):
        cj_4px_size.append(
            np.sum(
                np.where(
                    cj_4px == i,
                    1,
                    0
                )
            )
        )
        
        _lst[2] += cj_4px_size[-1]
        
        
    cnt += 1
    mito_cj_lst.append({folder : _lst})
    

In [ ]:
out_dir = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/cristae_junction_002/"

mito_cj_lst  = list()

connectivity = 26
cnt = 0

for folder in tqdm(sorted(os.listdir(out_dir))):
    if folder == ".DS_Store":
        continue
    cj_imgs, _ = load_Y_RGB(f"{out_dir}/{folder}", normalize=False)
    
    _lst = [0] * 3
    
    cj_2px = cc3d.connected_components((cj_imgs[:,:,:,0] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_2px) + 1):
        cj_2px_size.append(
            np.sum(
                np.where(
                    cj_2px == i,
                    1,
                    0
                )
            )
        )
        
        _lst[0] += cj_2px_size[-1]
        
    cj_3px = cc3d.connected_components((cj_imgs[:,:,:,1] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_3px) + 1):
        cj_3px_size.append(
            np.sum(
                np.where(
                    cj_3px == i,
                    1,
                    0
                )
            )
        )
        
        _lst[1] += cj_3px_size[-1]
        
    cj_4px = cc3d.connected_components((cj_imgs[:,:,:,2] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_4px) + 1):
        cj_4px_size.append(
            np.sum(
                np.where(
                    cj_4px == i,
                    1,
                    0
                )
            )
        )
        
        _lst[2] += cj_4px_size[-1]
        
        
    cnt += 1
    mito_cj_lst.append({folder : _lst})
    

In [ ]:
out_dir = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/cristae_junction_002/"

mito_cj_lst  = list()

connectivity = 26
cnt = 0

for folder in tqdm(sorted(os.listdir(out_dir))):
    if folder == ".DS_Store":
        continue
    cj_imgs, _ = load_Y_RGB(f"{out_dir}/{folder}", normalize=False)
    
    _lst = [0] * 3
    
    cj_2px = cc3d.connected_components((cj_imgs[:,:,:,0] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_2px) + 1):
        cj_2px_size.append(
            np.sum(
                np.where(
                    cj_2px == i,
                    1,
                    0
                )
            )
        )
        
        _lst[0] += cj_2px_size[-1]
        
    cj_3px = cc3d.connected_components((cj_imgs[:,:,:,1] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_3px) + 1):
        cj_3px_size.append(
            np.sum(
                np.where(
                    cj_3px == i,
                    1,
                    0
                )
            )
        )
        
        _lst[1] += cj_3px_size[-1]
        
    cj_4px = cc3d.connected_components((cj_imgs[:,:,:,2] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_4px) + 1):
        cj_4px_size.append(
            np.sum(
                np.where(
                    cj_4px == i,
                    1,
                    0
                )
            )
        )
        
        _lst[2] += cj_4px_size[-1]
        
        
    cnt += 1
    mito_cj_lst.append({folder : _lst})
    

In [ ]:
out_dir = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_junction_002/"

mito_cj_lst  = list()

connectivity = 26
cnt = 0

for folder in tqdm(sorted(os.listdir(out_dir))):
    if folder == ".DS_Store":
        continue
    cj_imgs, _ = load_Y_RGB(f"{out_dir}/{folder}", normalize=False)
    
    _lst = [0] * 3
    
    cj_2px = cc3d.connected_components((cj_imgs[:,:,:,0] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_2px) + 1):
        cj_2px_size.append(
            np.sum(
                np.where(
                    cj_2px == i,
                    1,
                    0
                )
            )
        )
        
        _lst[0] += cj_2px_size[-1]
        
    cj_3px = cc3d.connected_components((cj_imgs[:,:,:,1] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_3px) + 1):
        cj_3px_size.append(
            np.sum(
                np.where(
                    cj_3px == i,
                    1,
                    0
                )
            )
        )
        
        _lst[1] += cj_3px_size[-1]
        
    cj_4px = cc3d.connected_components((cj_imgs[:,:,:,2] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_4px) + 1):
        cj_4px_size.append(
            np.sum(
                np.where(
                    cj_4px == i,
                    1,
                    0
                )
            )
        )
        
        _lst[2] += cj_4px_size[-1]
        
        
    cnt += 1
    mito_cj_lst.append({folder : _lst})
    

In [ ]:
kd_2px_size = []
kd_3px_size = []
kd_4px_size = []

In [ ]:
out_dir = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_junction_002/"

mito_cj_lst  = list()

connectivity = 26
cnt = 0

for folder in tqdm(sorted(os.listdir(out_dir))):
    if folder == ".DS_Store":
        continue
    cj_imgs, _ = load_Y_RGB(f"{out_dir}/{folder}", normalize=False)
    
    _lst = [0] * 3
    
    cj_2px = cc3d.connected_components((cj_imgs[:,:,:,0] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_2px) + 1):
        kd_2px_size.append(
            np.sum(
                np.where(
                    cj_2px == i,
                    1,
                    0
                )
            )
        )
        
        _lst[0] += kd_2px_size[-1]
        
    cj_3px = cc3d.connected_components((cj_imgs[:,:,:,1] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_3px) + 1):
        kd_3px_size.append(
            np.sum(
                np.where(
                    cj_3px == i,
                    1,
                    0
                )
            )
        )
        
        _lst[1] += kd_3px_size[-1]
        
    cj_4px = cc3d.connected_components((cj_imgs[:,:,:,2] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_4px) + 1):
        kd_4px_size.append(
            np.sum(
                np.where(
                    cj_4px == i,
                    1,
                    0
                )
            )
        )
        
        _lst[2] += kd_4px_size[-1]
        
        
    cnt += 1
    mito_cj_lst.append({folder : _lst})
    

In [ ]:
out_dir = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_junction_002/"

mito_cj_lst  = list()

connectivity = 26
cnt = 0

for folder in tqdm(sorted(os.listdir(out_dir))):
    if folder == ".DS_Store":
        continue
    cj_imgs, _ = load_Y_RGB(f"{out_dir}/{folder}", normalize=False)
    
    _lst = [0] * 3
    
    cj_2px = cc3d.connected_components((cj_imgs[:,:,:,0] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_2px) + 1):
        kd_2px_size.append(
            np.sum(
                np.where(
                    cj_2px == i,
                    1,
                    0
                )
            )
        )
        
        _lst[0] += kd_2px_size[-1]
        
    cj_3px = cc3d.connected_components((cj_imgs[:,:,:,1] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_3px) + 1):
        kd_3px_size.append(
            np.sum(
                np.where(
                    cj_3px == i,
                    1,
                    0
                )
            )
        )
        
        _lst[1] += kd_3px_size[-1]
        
    cj_4px = cc3d.connected_components((cj_imgs[:,:,:,2] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_4px) + 1):
        kd_4px_size.append(
            np.sum(
                np.where(
                    cj_4px == i,
                    1,
                    0
                )
            )
        )
        
        _lst[2] += kd_4px_size[-1]
        
        
    cnt += 1
    mito_cj_lst.append({folder : _lst})
    

In [ ]:
out_dir = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/cristae_junction_002/"

mito_cj_lst  = list()

connectivity = 26
cnt = 0

for folder in tqdm(sorted(os.listdir(out_dir))):
    if folder == ".DS_Store":
        continue
    cj_imgs, _ = load_Y_RGB(f"{out_dir}/{folder}", normalize=False)
    
    _lst = [0] * 3
    
    cj_2px = cc3d.connected_components((cj_imgs[:,:,:,0] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_2px) + 1):
        kd_2px_size.append(
            np.sum(
                np.where(
                    cj_2px == i,
                    1,
                    0
                )
            )
        )
        
        _lst[0] += kd_2px_size[-1]
        
    cj_3px = cc3d.connected_components((cj_imgs[:,:,:,1] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_3px) + 1):
        kd_3px_size.append(
            np.sum(
                np.where(
                    cj_3px == i,
                    1,
                    0
                )
            )
        )
        
        _lst[1] += kd_3px_size[-1]
        
    cj_4px = cc3d.connected_components((cj_imgs[:,:,:,2] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_4px) + 1):
        kd_4px_size.append(
            np.sum(
                np.where(
                    cj_4px == i,
                    1,
                    0
                )
            )
        )
        
        _lst[2] += kd_4px_size[-1]
        
        
    cnt += 1
    mito_cj_lst.append({folder : _lst})
    

In [ ]:
out_dir = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/cristae_junction_002/"

mito_cj_lst  = list()

connectivity = 26
cnt = 0

for folder in tqdm(sorted(os.listdir(out_dir))):
    if folder == ".DS_Store":
        continue
    cj_imgs, _ = load_Y_RGB(f"{out_dir}/{folder}", normalize=False)
    
    _lst = [0] * 3
    
    cj_2px = cc3d.connected_components((cj_imgs[:,:,:,0] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_2px) + 1):
        kd_2px_size.append(
            np.sum(
                np.where(
                    cj_2px == i,
                    1,
                    0
                )
            )
        )
        
        _lst[0] += kd_2px_size[-1]
        
    cj_3px = cc3d.connected_components((cj_imgs[:,:,:,1] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_3px) + 1):
        kd_3px_size.append(
            np.sum(
                np.where(
                    cj_3px == i,
                    1,
                    0
                )
            )
        )
        
        _lst[1] += kd_3px_size[-1]
        
    cj_4px = cc3d.connected_components((cj_imgs[:,:,:,2] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_4px) + 1):
        kd_4px_size.append(
            np.sum(
                np.where(
                    cj_4px == i,
                    1,
                    0
                )
            )
        )
        
        _lst[2] += kd_4px_size[-1]
        
        
    cnt += 1
    mito_cj_lst.append({folder : _lst})
    

In [ ]:
out_dir = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/cristae_junction_002/"

mito_cj_lst  = list()

connectivity = 26
cnt = 0

for folder in tqdm(sorted(os.listdir(out_dir))):
    if folder == ".DS_Store":
        continue
    cj_imgs, _ = load_Y_RGB(f"{out_dir}/{folder}", normalize=False)
    
    _lst = [0] * 3
    
    cj_2px = cc3d.connected_components((cj_imgs[:,:,:,0] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_2px) + 1):
        kd_2px_size.append(
            np.sum(
                np.where(
                    cj_2px == i,
                    1,
                    0
                )
            )
        )
        
        _lst[0] += kd_2px_size[-1]
        
    cj_3px = cc3d.connected_components((cj_imgs[:,:,:,1] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_3px) + 1):
        kd_3px_size.append(
            np.sum(
                np.where(
                    cj_3px == i,
                    1,
                    0
                )
            )
        )
        
        _lst[1] += kd_3px_size[-1]
        
    cj_4px = cc3d.connected_components((cj_imgs[:,:,:,2] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_4px) + 1):
        kd_4px_size.append(
            np.sum(
                np.where(
                    cj_4px == i,
                    1,
                    0
                )
            )
        )
        
        _lst[2] += kd_4px_size[-1]
        
        
    cnt += 1
    mito_cj_lst.append({folder : _lst})
    

In [ ]:
cj_3px_size_ = np.array(cj_3px_size)
np.save("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/data/cristae_junction.npy", cj_3px_size_)

In [ ]:
kd_3px_size_ = np.array(kd_3px_size)
np.save("/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/data/cristae_junction.npy", kd_3px_size_)

In [ ]:
out_dir = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_junction_002/"

mito_cj_lst  = list()

connectivity = 26
cnt = 0

for folder in tqdm(sorted(os.listdir(out_dir))):
    if folder == ".DS_Store":
        continue
    cj_imgs, _ = load_Y_RGB(f"{out_dir}/{folder}", normalize=False)
    
    sum_cj = 0
    

        
    cj_3px = cc3d.connected_components((cj_imgs[:,:,:,1] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_3px) + 1):
        _count = np.sum(
                np.where(
                    cj_3px == i,
                    1,
                    0
                )
        )
        
        if _count > 1:
            sum_cj += _count
            

        
    cnt += 1
    mito_cj_lst.append({folder : sum_cj})
    
    
for node  in mito_cj_lst:
    print(list(node.values())[0])

In [ ]:
out_dir = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_junction_002/"

mito_cj_lst  = list()

connectivity = 26
cnt = 0

for folder in tqdm(sorted(os.listdir(out_dir))):
    if folder == ".DS_Store":
        continue
    cj_imgs, _ = load_Y_RGB(f"{out_dir}/{folder}", normalize=False)
    
    sum_cj = 0
    

        
    cj_3px = cc3d.connected_components((cj_imgs[:,:,:,1] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_3px) + 1):
        _count = np.sum(
                np.where(
                    cj_3px == i,
                    1,
                    0
                )
        )
        
        if _count > 1:
            sum_cj += _count
            

        
    cnt += 1
    mito_cj_lst.append({folder : sum_cj})
    
    
for node  in mito_cj_lst:
    print(list(node.values())[0])

In [ ]:
out_dir = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/cristae_junction_002/"

mito_cj_lst  = list()

connectivity = 26
cnt = 0

for folder in tqdm(sorted(os.listdir(out_dir))):
    if folder == ".DS_Store":
        continue
    cj_imgs, _ = load_Y_RGB(f"{out_dir}/{folder}", normalize=False)
    
    sum_cj = 0
    

        
    cj_3px = cc3d.connected_components((cj_imgs[:,:,:,1] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_3px) + 1):
        _count = np.sum(
                np.where(
                    cj_3px == i,
                    1,
                    0
                )
        )
        
        if _count > 1:
            sum_cj += _count
            

        
    cnt += 1
    mito_cj_lst.append({folder : sum_cj})
    
for node  in mito_cj_lst:
    print(list(node.values())[0])

In [ ]:
out_dir = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/cristae_junction_002/"

mito_cj_lst  = list()

connectivity = 26
cnt = 0

for folder in tqdm(sorted(os.listdir(out_dir))):
    if folder == ".DS_Store":
        continue
    cj_imgs, _ = load_Y_RGB(f"{out_dir}/{folder}", normalize=False)
    
    sum_cj = 0
    

        
    cj_3px = cc3d.connected_components((cj_imgs[:,:,:,1] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_3px) + 1):
        _count = np.sum(
                np.where(
                    cj_3px == i,
                    1,
                    0
                )
        )
        
        if _count > 1:
            sum_cj += _count
            

        
    cnt += 1
    mito_cj_lst.append({folder : sum_cj})
    
for node  in mito_cj_lst:
    print(list(node.values())[0])

In [ ]:
out_dir = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/cristae_junction_002/"

mito_cj_lst  = list()

connectivity = 26
cnt = 0

for folder in tqdm(sorted(os.listdir(out_dir))):
    if folder == ".DS_Store":
        continue
    cj_imgs, _ = load_Y_RGB(f"{out_dir}/{folder}", normalize=False)
    
    sum_cj = 0
    

        
    cj_3px = cc3d.connected_components((cj_imgs[:,:,:,1] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_3px) + 1):
        _count = np.sum(
                np.where(
                    cj_3px == i,
                    1,
                    0
                )
        )
        
        if _count > 1:
            sum_cj += _count
            

        
    cnt += 1
    mito_cj_lst.append({folder : sum_cj})
    
for node  in mito_cj_lst:
    print(list(node.values())[0])

In [ ]:
out_dir = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_junction_002/"

mito_cj_lst  = list()

connectivity = 26
cnt = 0

for folder in tqdm(sorted(os.listdir(out_dir))):
    if folder == ".DS_Store":
        continue
    cj_imgs, _ = load_Y_RGB(f"{out_dir}/{folder}", normalize=False)
    
    sum_cj = 0
    

        
    cj_3px = cc3d.connected_components((cj_imgs[:,:,:,1] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_3px) + 1):
        _count = np.sum(
                np.where(
                    cj_3px == i,
                    1,
                    0
                )
        )
        
        if _count > 1:
            sum_cj += _count
            

        
    cnt += 1
    mito_cj_lst.append({folder : sum_cj})
    
for node  in mito_cj_lst:
    print(list(node.values())[0])

In [ ]:
out_dir = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_junction_002/"

mito_cj_lst  = list()

connectivity = 26
cnt = 0

for folder in tqdm(sorted(os.listdir(out_dir))):
    if folder == ".DS_Store":
        continue
    cj_imgs, _ = load_Y_RGB(f"{out_dir}/{folder}", normalize=False)
    
    sum_cj = 0
    

        
    cj_3px = cc3d.connected_components((cj_imgs[:,:,:,1] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_3px) + 1):
        _count = np.sum(
                np.where(
                    cj_3px == i,
                    1,
                    0
                )
        )
        
        if _count > 1:
            sum_cj += _count
            

        
    cnt += 1
    mito_cj_lst.append({folder : sum_cj})
    
for node  in mito_cj_lst:
    print(list(node.values())[0])

In [ ]:
out_dir = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/cristae_junction_002/"

mito_cj_lst  = list()

connectivity = 26
cnt = 0

for folder in tqdm(sorted(os.listdir(out_dir))):
    if folder == ".DS_Store":
        continue
    cj_imgs, _ = load_Y_RGB(f"{out_dir}/{folder}", normalize=False)
    
    sum_cj = 0
    

        
    cj_3px = cc3d.connected_components((cj_imgs[:,:,:,1] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_3px) + 1):
        _count = np.sum(
                np.where(
                    cj_3px == i,
                    1,
                    0
                )
        )
        
        if _count > 1:
            sum_cj += _count
            

        
    cnt += 1
    mito_cj_lst.append({folder : sum_cj})
    
for node  in mito_cj_lst:
    print(list(node.values())[0])

In [ ]:
out_dir = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/cristae_junction_002/"

mito_cj_lst  = list()

connectivity = 26
cnt = 0

for folder in tqdm(sorted(os.listdir(out_dir))):
    if folder == ".DS_Store":
        continue
    cj_imgs, _ = load_Y_RGB(f"{out_dir}/{folder}", normalize=False)
    
    sum_cj = 0
    

        
    cj_3px = cc3d.connected_components((cj_imgs[:,:,:,1] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_3px) + 1):
        _count = np.sum(
                np.where(
                    cj_3px == i,
                    1,
                    0
                )
        )
        
        if _count > 1:
            sum_cj += _count
            

        
    cnt += 1
    mito_cj_lst.append({folder : sum_cj})
    
for node  in mito_cj_lst:
    print(list(node.values())[0])

In [ ]:
out_dir = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/cristae_junction_002/"

mito_cj_lst  = list()

connectivity = 26
cnt = 0

for folder in tqdm(sorted(os.listdir(out_dir))):
    if folder == ".DS_Store":
        continue
    cj_imgs, _ = load_Y_RGB(f"{out_dir}/{folder}", normalize=False)
    
    sum_cj = 0
    

        
    cj_3px = cc3d.connected_components((cj_imgs[:,:,:,1] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_3px) + 1):
        _count = np.sum(
                np.where(
                    cj_3px == i,
                    1,
                    0
                )
        )
        
        if _count > 1:
            sum_cj += _count
            

        
    cnt += 1
    mito_cj_lst.append({folder : sum_cj})
    
for node  in mito_cj_lst:
    print(list(node.values())[0])

In [ ]:
out_dir = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_junction_002/"

mito_cj_lst  = list()

connectivity = 26
cnt = 0

for folder in tqdm(sorted(os.listdir(out_dir))):
    if folder == ".DS_Store":
        continue
    cj_imgs, _ = load_Y_RGB(f"{out_dir}/{folder}", normalize=False)
    
    sum_cj = 0
    

        
    cj_3px = cc3d.connected_components((cj_imgs[:,:,:,1] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_3px) + 1):
        _count = np.sum(
                np.where(
                    cj_3px == i,
                    1,
                    0
                )
        )
        
        if _count > 1:
            sum_cj += 1
            

        
    cnt += 1
    mito_cj_lst.append({folder : sum_cj})
    
    
for node  in mito_cj_lst:
    print(list(node.values())[0])

In [ ]:
out_dir = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_junction_002/"

mito_cj_lst  = list()

connectivity = 26
cnt = 0

for folder in tqdm(sorted(os.listdir(out_dir))):
    if folder == ".DS_Store":
        continue
    cj_imgs, _ = load_Y_RGB(f"{out_dir}/{folder}", normalize=False)
    
    sum_cj = 0
    

        
    cj_3px = cc3d.connected_components((cj_imgs[:,:,:,1] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_3px) + 1):
        _count = np.sum(
                np.where(
                    cj_3px == i,
                    1,
                    0
                )
        )
        
        if _count > 1:
            sum_cj += 1
            

        
    cnt += 1
    mito_cj_lst.append({folder : sum_cj})
    
    
for node  in mito_cj_lst:
    print(list(node.values())[0])

In [ ]:
out_dir = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/cristae_junction_002/"

mito_cj_lst  = list()

connectivity = 26
cnt = 0

for folder in tqdm(sorted(os.listdir(out_dir))):
    if folder == ".DS_Store":
        continue
    cj_imgs, _ = load_Y_RGB(f"{out_dir}/{folder}", normalize=False)
    
    sum_cj = 0
    

        
    cj_3px = cc3d.connected_components((cj_imgs[:,:,:,1] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_3px) + 1):
        _count = np.sum(
                np.where(
                    cj_3px == i,
                    1,
                    0
                )
        )
        
        if _count > 1:
            sum_cj += 1
            

        
    cnt += 1
    mito_cj_lst.append({folder : sum_cj})
    
    
for node  in mito_cj_lst:
    print(list(node.values())[0])

In [ ]:
out_dir = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/cristae_junction_002/"

mito_cj_lst  = list()

connectivity = 26
cnt = 0

for folder in tqdm(sorted(os.listdir(out_dir))):
    if folder == ".DS_Store":
        continue
    cj_imgs, _ = load_Y_RGB(f"{out_dir}/{folder}", normalize=False)
    
    sum_cj = 0
    

        
    cj_3px = cc3d.connected_components((cj_imgs[:,:,:,1] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_3px) + 1):
        _count = np.sum(
                np.where(
                    cj_3px == i,
                    1,
                    0
                )
        )
        
        if _count > 1:
            sum_cj += 1
            

        
    cnt += 1
    mito_cj_lst.append({folder : sum_cj})
    
    
for node  in mito_cj_lst:
    print(list(node.values())[0])

In [ ]:
out_dir = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/cristae_junction_002/"

mito_cj_lst  = list()

connectivity = 26
cnt = 0

for folder in tqdm(sorted(os.listdir(out_dir))):
    if folder == ".DS_Store":
        continue
    cj_imgs, _ = load_Y_RGB(f"{out_dir}/{folder}", normalize=False)
    
    sum_cj = 0
    

        
    cj_3px = cc3d.connected_components((cj_imgs[:,:,:,1] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_3px) + 1):
        _count = np.sum(
                np.where(
                    cj_3px == i,
                    1,
                    0
                )
        )
        
        if _count > 1:
            sum_cj += 1
            

        
    cnt += 1
    mito_cj_lst.append({folder : sum_cj})
    
    
for node  in mito_cj_lst:
    print(list(node.values())[0])

In [ ]:
out_dir = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_junction_002/"

mito_cj_lst  = list()

connectivity = 26
cnt = 0

for folder in tqdm(sorted(os.listdir(out_dir))):
    if folder == ".DS_Store":
        continue
    cj_imgs, _ = load_Y_RGB(f"{out_dir}/{folder}", normalize=False)
    
    sum_cj = 0
    

        
    cj_3px = cc3d.connected_components((cj_imgs[:,:,:,1] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_3px) + 1):
        _count = np.sum(
                np.where(
                    cj_3px == i,
                    1,
                    0
                )
        )
        
        if _count > 1:
            sum_cj += 1
            

        
    cnt += 1
    mito_cj_lst.append({folder : sum_cj})
    
    
for node  in mito_cj_lst:
    print(list(node.values())[0])

In [ ]:
out_dir = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_junction_002/"

mito_cj_lst  = list()

connectivity = 26
cnt = 0

for folder in tqdm(sorted(os.listdir(out_dir))):
    if folder == ".DS_Store":
        continue
    cj_imgs, _ = load_Y_RGB(f"{out_dir}/{folder}", normalize=False)
    
    sum_cj = 0
    

        
    cj_3px = cc3d.connected_components((cj_imgs[:,:,:,1] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_3px) + 1):
        _count = np.sum(
                np.where(
                    cj_3px == i,
                    1,
                    0
                )
        )
        
        if _count > 1:
            sum_cj += 1
            

        
    cnt += 1
    mito_cj_lst.append({folder : sum_cj})
    
    
for node  in mito_cj_lst:
    print(list(node.values())[0])

In [ ]:
out_dir = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/cristae_junction_002/"

mito_cj_lst  = list()

connectivity = 26
cnt = 0

for folder in tqdm(sorted(os.listdir(out_dir))):
    if folder == ".DS_Store":
        continue
    cj_imgs, _ = load_Y_RGB(f"{out_dir}/{folder}", normalize=False)
    
    sum_cj = 0
    

        
    cj_3px = cc3d.connected_components((cj_imgs[:,:,:,1] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_3px) + 1):
        _count = np.sum(
                np.where(
                    cj_3px == i,
                    1,
                    0
                )
        )
        
        if _count > 1:
            sum_cj += 1
            

        
    cnt += 1
    mito_cj_lst.append({folder : sum_cj})
    
    
for node  in mito_cj_lst:
    print(list(node.values())[0])

In [ ]:
out_dir = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/cristae_junction_002/"

mito_cj_lst  = list()

connectivity = 26
cnt = 0

for folder in tqdm(sorted(os.listdir(out_dir))):
    if folder == ".DS_Store":
        continue
    cj_imgs, _ = load_Y_RGB(f"{out_dir}/{folder}", normalize=False)
    
    sum_cj = 0
    

        
    cj_3px = cc3d.connected_components((cj_imgs[:,:,:,1] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_3px) + 1):
        _count = np.sum(
                np.where(
                    cj_3px == i,
                    1,
                    0
                )
        )
        
        if _count > 1:
            sum_cj += 1
            

        
    cnt += 1
    mito_cj_lst.append({folder : sum_cj})
    
    
for node  in mito_cj_lst:
    print(list(node.values())[0])

In [ ]:
out_dir = "/home/suga/drobo/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/cristae_junction_002/"

mito_cj_lst  = list()

connectivity = 26
cnt = 0

for folder in tqdm(sorted(os.listdir(out_dir))):
    if folder == ".DS_Store":
        continue
    cj_imgs, _ = load_Y_RGB(f"{out_dir}/{folder}", normalize=False)
    
    sum_cj = 0
    

        
    cj_3px = cc3d.connected_components((cj_imgs[:,:,:,1] == 1).astype(int), connectivity = connectivity)
    for i in range(1, np.max(cj_3px) + 1):
        _count = np.sum(
                np.where(
                    cj_3px == i,
                    1,
                    0
                )
        )
        
        if _count > 1:
            sum_cj += 1
            

        
    cnt += 1
    mito_cj_lst.append({folder : sum_cj})
    
    
for node  in mito_cj_lst:
    print(list(node.values())[0])